# MoviePlumeField Demo (Video → Zarr → Env)

This notebook demonstrates a lightweight, end-to-end workflow for using a movie-backed plume field.
It covers: (1) dataset schema and calibration notes, (2) creating/reading a tiny Zarr dataset,
(3) previewing frames via xarray, (4) instantiating a minimal MoviePlumeField adapter,
and (5) stepping the environment a few steps while updating the plume each step, with a small plot.

The demo intentionally keeps dependencies optional: if xarray/zarr are unavailable, it falls back
to in-memory frames so you can still follow the structure.


## Dataset Schema and Calibration

Recommended schema for movie-derived plume datasets:
- Variable: `concentration` with dims `(t, y, x)` and dtype `float32` in [0, 1].
- Chunking: approx `(8, 64, 64)` with a compressed Zarr codec (e.g., Blosc/Zstd).
- Key attributes (stored in `attrs`):
  - `schema_version`: e.g., `v0`
  - `fps`: frames per second (float)
  - `timebase`: e.g., `seconds`
  - `pixel_to_grid`: scale factor (grid cells per pixel), default 1.0
  - `origin`: `(y0, x0)` in grid coordinates for top-left pixel (default (0, 0))
  - `extent`: optional physical extent metadata if applicable
  - `source_dtype`: original source dtype before normalization
  - `provenance`: free-form or structured manifest link

Calibration overview:
- Mapping is from image pixels `(y, x)` to environment grid `(y, x)` after applying `pixel_to_grid`.
- The top-left pixel maps to `origin=(y0, x0)`. Increasing `x` moves right; increasing `y` moves down.
- If your camera has different axes (e.g., origin bottom-left or transposed frames), encode that in attrs
  and apply transforms at load time to ensure `(t, y, x)` and correct orientation in grid coordinates.


In [ ]:
# Imports and helpers
from __future__ import annotations

from contextlib import suppress
from pathlib import Path
import math
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

# Optional dependencies
try:
    import xarray as xr  # type: ignore

    _HAVE_XARRAY = True
except Exception:
    xr = None  # type: ignore
    _HAVE_XARRAY = False

# Environment components (manual wiring)
from plume_nav_sim.core.geometry import GridSize, Coordinates
from plume_nav_sim.envs.component_env import ComponentBasedEnvironment
from plume_nav_sim.actions.discrete_grid import DiscreteGridActions
from plume_nav_sim.observations.concentration import ConcentrationSensor
from plume_nav_sim.rewards.step_penalty import StepPenaltyReward
from plume_nav_sim.plume.concentration_field import ConcentrationField

RESULTS = Path("results")
RESULTS.mkdir(parents=True, exist_ok=True)
ZARR_PATH = RESULTS / "movie_plume_demo.zarr"


def _inline_show(fig: plt.Figure) -> None:
    """Headless-safe display helper."""
    display(fig)
    plt.close(fig)


def _make_moving_gaussian_frames(T=12, H=32, W=32, sigma=6.0):
    ys = np.arange(H)[:, None]
    xs = np.arange(W)[None, :]
    frames = []
    for t in range(T):
        cx = int((t / max(1, T - 1)) * (W - 1))
        cy = int((t / max(1, T - 1)) * (H - 1))
        g = np.exp(-(((xs - cx) ** 2 + (ys - cy) ** 2) / (2.0 * sigma**2))).astype(
            np.float32
        )
        frames.append(g)
    arr = np.stack(frames, axis=0)  # (t, y, x)
    # Normalize strictly to [0, 1]
    vmin, vmax = float(arr.min()), float(arr.max())
    if vmax > vmin:
        arr = (arr - vmin) / (vmax - vmin)
    return arr.astype(np.float32)


def _ensure_tiny_zarr(out_path: Path) -> tuple[np.ndarray, dict]:
    """Create a tiny demo Zarr store if xarray is available; otherwise return frames in-memory.
    Returns (frames, attrs).
    """
    frames = _make_moving_gaussian_frames(T=12, H=32, W=32, sigma=6.0)
    attrs = {
        "schema_version": "v0",
        "fps": 5.0,
        "timebase": "seconds",
        "pixel_to_grid": 1.0,
        "origin": (0, 0),
        "extent": None,
        "source_dtype": "float32",
        "provenance": {"source": "synthetic", "note": "demo"},
    }
    if _HAVE_XARRAY:
        # Write a tiny Zarr for illustration
        da = xr.DataArray(
            frames, dims=("t", "y", "x"), name="concentration", attrs=attrs
        )
        ds = xr.Dataset({"concentration": da})
        if out_path.exists():
            import shutil

            with suppress(Exception):
                shutil.rmtree(out_path)
        try:
            ds.to_zarr(out_path, mode="w")
        except Exception as e:
            print("Could not write Zarr store (continuing with in-memory frames):", e)
    return frames, attrs


frames, ds_attrs = _ensure_tiny_zarr(ZARR_PATH)
frames.shape, list(ds_attrs.items())[:3]  # quick peek

In [ ]:
# Preview first frame via xarray if available (fallback: matplotlib imshow from numpy)
if _HAVE_XARRAY and ZARR_PATH.exists():
    ds = xr.open_zarr(ZARR_PATH)
    ax = ds["concentration"].isel(t=0).plot.imshow(cmap="magma")
    _ = ax.axes.set_title("t=0 (xarray preview)")
    _inline_show(ax.figure)
else:
    fig, ax = plt.subplots()
    im = ax.imshow(frames[0], cmap="magma")
    ax.set_title("t=0 (numpy preview)")
    fig.colorbar(im, ax=ax)
    _inline_show(fig)

In [ ]:
# Minimal MoviePlumeField adapter using the generated frames
class MoviePlumeField:
    def __init__(self, frames: np.ndarray, attrs: dict):
        assert (
            frames.ndim == 3 and frames.shape[1] > 0 and frames.shape[2] > 0
        ), "frames must be (t, y, x)"
        self.frames = frames.astype(np.float32, copy=False)
        self.attrs = dict(attrs)
        self.t = 0

    @property
    def n_frames(self) -> int:
        return int(self.frames.shape[0])

    @property
    def grid_wh(self) -> tuple[int, int]:
        return int(self.frames.shape[2]), int(self.frames.shape[1])  # (W, H)

    def frame(self, t: int | None = None) -> np.ndarray:
        idx = self.t if t is None else int(t)
        idx = max(0, min(idx, self.n_frames - 1))
        return self.frames[idx]

    def advance(self, step: int = 1) -> int:
        self.t = (self.t + int(step)) % self.n_frames
        return self.t


movie = MoviePlumeField(frames, ds_attrs)
movie.n_frames, movie.grid_wh

In [ ]:
# Wire a component-based environment and step a few times, updating the plume each step
W, H = movie.grid_wh
grid = GridSize(width=W, height=H)
goal = Coordinates(W // 2, H // 2)

# Start with the first frame as the concentration field
field = ConcentrationField(grid_size=grid)
field.field_array = movie.frame(0)  # 2D np.ndarray [H, W] in [0,1]
field.is_generated = True

actions = DiscreteGridActions(step_size=1)
sensor = ConcentrationSensor()
reward = StepPenaltyReward(
    goal_position=goal, goal_radius=3.0, goal_reward=1.0, step_penalty=0.01
)
env = ComponentBasedEnvironment(
    action_processor=actions,
    observation_model=sensor,
    reward_function=reward,
    concentration_field=field,
    grid_size=grid,
    goal_location=goal,
    max_steps=50,
)
obs, info = env.reset(seed=123)
path = [tuple(info.get("agent_position", (W // 2, H // 2)))]
for i in range(10):
    # Update the environment's concentration field with the next movie frame
    env._concentration_field.field_array = movie.frame(i)  # simple, effective for demo
    obs, r, term, trunc, info = env.step(env.action_space.sample())
    path.append(tuple(info.get("agent_position", path[-1])))
    if term or trunc:
        break

# Plot the latest frame with path overlay
fig, ax = plt.subplots(figsize=(4, 4))
im = ax.imshow(movie.frame(None), cmap="magma", origin="upper")
xs = [p[0] for p in path]
ys = [p[1] for p in path]
ax.plot(xs, ys, color="cyan", marker="o", linewidth=1)
ax.set_title("Agent path over last frame")
fig.colorbar(im, ax=ax)
_inline_show(fig)

## CI Smoke Option

A small script suitable for CI smoke is provided at:
- `src/backend/examples/movie_plume_smoke.py`

It generates a tiny Zarr (if xarray/zarr are available), loads it, checks basic shape/min/max,
and exits. You can invoke it in a job without requiring display backends.
